# Transofrm the dervied data

## Imports

In [26]:
import praw 
import pandas as pd
import matplotlib as m
from dotenv import load_dotenv
from pathlib import Path
import os

## Reading from Bronze Layer

In [27]:
BASE_DIR = Path().resolve().parent  

bronze_file_path = BASE_DIR / 'data' / 'bronze' / 'raw_reddit_posts.csv'

if bronze_file_path.exists():
    subreddit_df = pd.read_csv(bronze_file_path)
    print("Path successfully read")
else:
    print(f"Path {bronze_file_path} not found!")

# print(subreddit_df)

Path successfully read


## Clean the data

### Convert date to readable format

### Check for nulls

In [30]:
if subreddit_df.isnull().values.any():
    subreddit_df.dropna(inplace=True)
    print("Null removed successfully")
else:
    print("No nulls were found!")

    #functionality for capturing the null cols

No nulls were found!


### Filter out irrelevant posts

### Remove unwanted characters

## Load to Silver Layer